In [2]:
!pip install transformers

     |████████████████████████████████| 1.3MB 6.1MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 14.5MB/s 
     |████████████████████████████████| 890kB 42.7MB/s 
     |████████████████████████████████| 2.9MB 43.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a249846212ad4f39fa5b175f2ec8c86c08842e945f02d6e4fe312e924934de42
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [16]:
import os
import random

from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import BertTokenizer, BertForPreTraining, BertForMaskedLM, Trainer, TrainingArguments
import torch

import pandas as pd

In [ ]:
# df_rf = pd.read_csv('./../../../russianfood/russianfood_filtred.csv')
# df_er = pd.read_csv('./../../../eda_ru_dataset/eda_ru.csv', sep=';')

# if os.path.isfile(filename):
#     os.remove(filename)

# with open(filename, 'a') as file:
#     for line in df_rf['title']:
#         file.write(line + '\n')
        
#     for line in df_er['title']:
#         file.write(line + '\n')
    
#     for steps in df_er['steps'].dropna():
#         for step in steps.split('|'):
#             step = step.split('. ', maxsplit=1)
#             if len(step) >= 2:
#                 file.write(step[1] + '\n')
            
#     for steps in df_rf['steps'].dropna():
#         for step in steps.split('|'):
#             file.write(step + '\n')

In [25]:
filename = 'texts.txt'

In [26]:
if not os.path.exists('GDriveDL.py'):
    !wget https://raw.githubusercontent.com/matthuisman/gdrivedl/master/gdrivedl.py -O GDriveDL.py

if not os.path.exists(filename):
    !python GDriveDL.py https://drive.google.com/file/d/1BZJ3VH-Bm7daRTko7QMC0x6M2LwwxVvP/view?usp=sharing .

--2020-11-04 18:53:13--  https://raw.githubusercontent.com/matthuisman/gdrivedl/master/gdrivedl.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6773 (6.6K) [text/plain]
Saving to: ‘GDriveDL.py’

GDriveDL.py         100%[===================>]   6.61K  --.-KB/s    in 0s      

2020-11-04 18:53:14 (43.5 MB/s) - ‘GDriveDL.py’ saved [6773/6773]

./texts.txt
[==================================================] 70.95MB/70.95MB


In [31]:
with open(filename,'r') as source:
    lines = source.readlines()

random.shuffle(lines)

with open(filename,'w') as target:
    target.writelines(lines)

In [18]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/text2image'

Mounted at /content/drive


In [32]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
rubert = BertForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased")

Some weights of BertForMaskedLM were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset = LineByLineTextDataset(file_path=filename, block_size=512, tokenizer=tokenizer)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.25
)

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
training_args = TrainingArguments(
    output_dir=f'{drive_path}/rubert_finetuned_results',          # output directory
    num_train_epochs=1,              # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    prediction_loss_only=True,
    save_steps=2000,
    save_total_limit=3,
)

In [ ]:
trainer = Trainer(
    args=training_args,
    model=rubert, 
    train_dataset=dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [14]:
torch.cuda.empty_cache()